In [ ]:
pip install sentencepiece
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content  # <-- Add this line
    soup = BeautifulSoup(html_content, 'html.parser')
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    paragraphs = main_content.find_all('p')
    print(paragraphs)
    text_content = ' '.join([p.text for p in paragraphs])

    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    summary = summarization_pipeline(text_content, max_length=150, min_length=30, do_sample=False)

    # Return the summary as a string
    return summary[0]['summary_text']


url = 'https://indiankanoon.org/doc/168100369/'
summary = summarize_legal_document(url)
print(summary)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[<p>Take notes as you read a judgment using our <b>Virtual Legal Assistant</b> and get email alerts whenever a new judgment matches your query (<b>Query Alert Service</b>). Try out our <b>Premium Member</b> services -- <a href="/members/">Free for one month</a>.</p>, <p id="p_1">           1.    Leave granted.
</p>, <p id="p_2">           2.    This appeal has been  filed  impugning  the  judgment  and
           order dated 12th November,  2010  in  FAO(OS)  NO.607  of  2010,
           whereby the Division Bench of the Delhi High Court dismissed the
           appeal filed by the appellant in I.A.  NO.13902  of  2008  filed
           under Order VIII rule 1 praying for extension of time for filing
           written statement by the defendant  i.e.  the  appellant  herein
           till I.A. No. 11803 of 2008 filed  under  Order  I  rule  10  to
           implead Mr. Sharad Maheshwari as plaintiff.  The  aforesaid  Mr.
           Sharad Maheshwari is the  Managing  Director  of  t

Token indices sequence length is longer than the specified maximum sequence length for this model (2389 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

The above model is not able to handle documents of such a large size

In [ ]:
def summarize_legal_document(url, max_length=800, min_length=50, num_beams=4):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])

    # Load the summarization model and tokenizer
    model_name = "sshleifer/distilbart-xsum-12-1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Tokenize the input text and generate the summary
    inputs = tokenizer.encode(text_content, return_tensors='pt', max_length=max_length, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, num_beams=num_beams)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Return the summary as a string
    return summary

url = 'https://indiankanoon.org/doc/105912122/'
summary = summarize_legal_document(url)
print(summary)


 The Supreme Court in India has heard a petition calling for changes to the way the way it deals with the dishonor of cheques in the country, which are still being held in a case against a number of cases involving involving banks and banks.


It generated the summary but not too proper

# Considering document length for setting max_length hyperparameter


In [ ]:
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    document_length = len(text_content)
    if document_length < 1000:
        max_length = 100
    elif document_length < 2000:
        max_length = 150
    elif document_length < 3000:
        max_length = 200
    else:
        max_length = 250
    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    try:
        summary = summarization_pipeline(text_content, max_length=max_length, min_length=30, do_sample=False, truncation=True)
    except ValueError as e:
        return f"Error: {e}"

    # Return the summary as a string
    return summary[0]['summary_text']

# Example usage
url = 'https://indiankanoon.org/doc/1923667/'
summary = summarize_legal_document(url)
print(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 The Ratlam municipal town, like many Indian urban centers, is populous with human and sub-human species . The rich have bungalows and toilets, the poor live on pavements and litter the street with human excreta because they use roadsides as latrines . The city fathers being too busy with other issues to bother about the human condition, cesspools and stinks, dirtied the place beyond endurance which made the well-to-do citizens protest, but the crying demand for basic sanitation and public drains fell on deaf ears .


We considered using the default model with static and dynamic max_length

In [ ]:
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"

    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    try:
        summary = summarization_pipeline(text_content, max_length=512, min_length=30, do_sample=False, truncation=True)
    except ValueError as e:
        return f"Error: {e}"

    # Return the summary as a string
    return summary[0]['summary_text']

# Example usage
url = 'https://indiankanoon.org/doc/105912122/'
summary = summarize_legal_document(url)
print(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Indian Banks’ Association (IBA) along with Punjab . National Bank and another, seeking the following reliefs :- Laying down appropriate guidelines/directions to be followed  by all . Courts within the territory of India competent to try  a complaint. under Section 138 of the Negotiable Instruments Act, 1881 . Issue a writ of mandamus directing  the  respondents,  to adopt  necessary policy and legislative changes to deal with cases relating to dishonor of cheques .


In [ ]:
url = 'https://indiankanoon.org/doc/1072049/'
summary = summarize_legal_document(url)
print(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Appeal from the High Court of Judicature, Bombay, in a reference under section 66 of the Indian Income-tax . The company had paid Rs. 1,22,675 as property tax and Rs. 32,760 as urban property tax . The High Court answered all three questions in the negative and hence this appeal .


# Code for generating summaries as a csv file

In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    document_length = len(text_content)
    if document_length < 1000:
        max_length = 100
    elif document_length < 2000:
        max_length = 150
    elif document_length < 3000:
        max_length = 200
    else:
        max_length = 250
    # Use the Hugging Face Summarization pipeline to generate a summary
    summarization_pipeline = pipeline("summarization")
    try:
        summary = summarization_pipeline(text_content, max_length=max_length, min_length=30, do_sample=False, truncation=True)
    except ValueError as e:
        return f"Error: {e}"

    # Return the summary as a string
    return summary[0]['summary_text']

# Example usage
path = 'Case Links-20230417T110231Z-001/Case Links/'
file = open(path+'tenant/urls.txt','r')
df = pd.DataFrame(columns=['Case Number','Summary'])
count=0
for url in file:
    summary = summarize_legal_document(url.strip())
    case_num = url.split('/')[-2]
#     file = open(path+'1947/'+case_num+'.txt','w')
#     file.write(summary)
#     file.close()
    df = pd.concat([pd.DataFrame([[case_num,summary]],columns=df.columns), df], ignore_index=True)
    df.to_csv(path+'tenant/summary.csv',index=False)
    count = count+1
    if count%10 == 0:
        print(count)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

10


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

20


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

...

2920


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

2930


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleife

2940


It is taking around 20 min to summarize 100 documents

# nsi319/legal-led-base-16384 - HugginFace model gives summary in more legal-domain specific language and also repeated sentences, so we are using a default model (sshleifer/distilbart-cnn-12-6) which is giving summaries which even a layman can understand and also not repeating the sentences
# The legal-led-base-16384 model was trained on sec-litigation-releases (US) dataset consisting more than 2700 litigation releases and complaints. So it is adding some extra points related to some US cases in the summary even if the document context is Indian

In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384")

padding = "max_length"
def summarize_legal_document(url):
    # Retrieve the HTML content of the legal document
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the main text content of the legal document
    main_content = soup.find("div", {"class": "judgments"})
    if main_content is None:
        return "Error: Could not find main content"
    paragraphs = main_content.find_all('p')
    text_content = ' '.join([p.text for p in paragraphs])
    if not text_content:
        return "Error: Could not extract text content"
    input_tokenized = tokenizer.encode(text_content, return_tensors='pt',padding=padding,pad_to_max_length=True, max_length=6144,truncation=True)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=4,
                                  no_repeat_ngram_size=3,
                                  length_penalty=3,
                                  min_length=350,
                                  max_length=1000)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    return summary

# Example usage
path = 'Case Links-20230417T110231Z-001/Case Links/'
file = open(path+'1947/urls_1947.txt','r')
df = pd.DataFrame(columns=['Case Number','Summary'])
count=0
# for url in file:
url = 'https://indiankanoon.org/doc/1923667/'
summary = summarize_legal_document(url.strip())
print(summary)
#     break
#     case_num = url.split('/')[-2]
#     file = open(path+'1947/'+case_num+'.txt','w')
#     file.write(summary)
#     file.close()
#     df = pd.concat([pd.DataFrame([[case_num,summary]],columns=df.columns), df], ignore_index=True)
#     df.to_csv(path+'1947/summary_1947.csv',index=False)
#     count = count+1
#     if count%10 == 0:
#         print(count)


The U.S. District Court for the Western District of New York yesterday entered a final judgment against Ratlam Municipality for violating the antifraud provisions of Section 133 of the Municipal Code.  According to the court's order, filed in the United States District Court in New York, Ratlam Municipal Corporation ("Municipality") has agreed to the entry of an order requiring it to construct drains and sewers at a time when it has no drainage system.  The court found that the M.P. Municipality had failed to adequately deal with the issue of public nuisance.  As alleged in the court order, the municipal council and its executive officers spent half a million dollars on cleaning up the street and constructing the drains by rousing the residents of the area and laying out the city's limited financial resources.  Without admitting or denying the allegations of the court, the Municipal Council and the executive officers have agreed to settle the matter by consenting to a judgment that per

We tried using nsi319/legal-led-base-16384 model which is trained specifically on legal documents